In [1]:
!git clone https://github.com/scorrea92/twitter-sentiment-analysis.git

Cloning into 'twitter-sentiment-analysis'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 193 (delta 112), reused 131 (delta 54), pack-reused 0
Receiving objects: 100% (193/193), 5.79 MiB | 19.83 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [0]:
!mv twitter-sentiment-analysis/* .

In [4]:
!unzip personal_library.zip

Archive:  personal_library.zip
   creating: personal_library/
  inflating: personal_library/.pylintrc  
   creating: personal_library/test/
 extracting: personal_library/test/.keep  
   creating: personal_library/test/NLP/
   creating: personal_library/test/NLP/bert/
  inflating: personal_library/test/NLP/bert/tokenization_test.py  
  inflating: personal_library/__init__.py  
   creating: personal_library/sce_keras/
  inflating: personal_library/sce_keras/eval_tools.py  
  inflating: personal_library/sce_keras/loss_functions.py  
  inflating: personal_library/sce_keras/.DS_Store  
   creating: personal_library/sce_keras/core/
 extracting: personal_library/sce_keras/core/__init__.py  
   creating: personal_library/sce_keras/core/__pycache__/
  inflating: personal_library/sce_keras/core/__pycache__/utils.cpython-36.pyc  
  inflating: personal_library/sce_keras/core/__pycache__/__init__.cpython-36.pyc  
  inflating: personal_library/sce_keras/core/generators.py  
  inflating: personal_lib

In [0]:
!mkdir model_wehigts

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
!ls /content/gdrive/My\ Drive

 6mesesbajareso.mp4	     Planeación_Almuerzoz.gsheet
'Colab Notebooks'	     TDV
'Gastos de la casa.gsheet'  'TFM Sebastian Correa (shared)'
 model_wehigts


In [0]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Dense,
    Input,
    LSTM,
    Embedding,
    Dropout,
    Activation,
    SpatialDropout1D
)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from ast import literal_eval
from sklearn.model_selection import train_test_split
from tensorflow.python.keras import backend as K


sess = tf.Session()
K.set_session(sess)

In [0]:
df = pd.read_csv('data/pandas_data_frame.csv', index_col=0)
all_data = df.where((pd.notnull(df)), '')
all_data['hashtag'] = all_data['hashtag'].apply(literal_eval)

full_text = all_data['tidy_tweet'][(all_data['label']=='1.0') | (all_data['label']=='0.0')]
y = all_data['label'][(all_data['label']=='1.0') | (all_data['label']=='0.0')]

In [0]:
max_len = 120

texts = full_text.tolist()
texts = [' '.join(t.split()[:max_len]) for t in texts]
texts = np.array(texts, dtype=object)[:, np.newaxis]

In [11]:
x_train, x_val, y_train, y_val = train_test_split(texts, y, random_state=1992, test_size=0.2)

print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)

(25569, 1) (25569,)
(6393, 1) (6393,)


In [18]:
import sys
sys.path.append("../") 
from personal_library.sce_keras.loss_functions import f1_loss
from personal_library.sce_keras.metrics_functions import f1
from personal_library.sce_keras.layers.elmo import ElmoEmbeddingLayer
from personal_library.sce_keras.callbacks import (
    LearningRateDecay,
    WarmUpCosineDecayScheduler
)


num_classes = 1
batch_size = 512
epochs = 200
learnRate = 0.001
lstm_out = 200
warmup_epoch = 20

warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=learnRate,
                                        warmup_learning_rate=0,
                                        warmup_epoch=warmup_epoch,
                                        hold_base_rate_steps=5,
                                        verbose=0)

checkpoint_path = "/content/gdrive/My Drive/model_wehigts/8_w.hdf5"
checkpoint_path1 = "/content/gdrive/My Drive/model_wehigts/8_ch.hdf5"
checkpointer = ModelCheckpoint(filepath=checkpoint_path,
                               monitor='val_f1', verbose=2,
                               save_best_only=True, mode='max')
checkpointer1 = ModelCheckpoint(filepath=checkpoint_path1,
                               monitor='val_loss', verbose=2,
                               save_best_only=False, mode='min')

input_text = Input(shape=(1,), dtype="string", name='input_0')
x = ElmoEmbeddingLayer(trainable=False)(input_text)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dense(num_classes, activation="sigmoid")(x)

model = Model(inputs=[input_text], outputs=x)

model.summary()

# 'binary_crossentropy'
model.compile(loss='binary_crossentropy',#f1_loss, 
              optimizer='adam', 
              metrics=['accuracy', f1])

sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

history = model.fit(x_train, y_train, 
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[checkpointer, checkpointer1, warm_up_lr]
                   )

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
elmo_embedding_layer_4 (Elmo (None, 1024)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________
Train on 25569 samples, validate on 6393 samples
Epoch 1/200
25088/25569 [============================>.] - ETA: 1s - l

In [19]:
from sklearn.metrics import f1_score, accuracy_score

#Load best model
model.load_weights(checkpoint_path)
y_pred = model.predict(x_val, batch_size=1)
y_pred = np.where(y_pred > 0.5, 1, 0)

print("Own emmbeding f1_sklearn: {}".format(f1_score(y_val.astype(float), y_pred)))
print("Own emmbeding accuracy: {}".format(accuracy_score(y_val.astype(float), y_pred)))

Own emmbeding f1_sklearn: 0.5683962264150944
Own emmbeding accuracy: 0.9427498826841858
